# Image classification

Notebook to produce the classification of regions within the images in northern scotland based on our training data.

+ We will start by loading in the classification data, as [lat,lon] coordinates and a classification.
+ We will also load in the GEE image, and extracting the appropriate bands, and creating new ones (NDWI, etc) if appropriate.

+ We will have to split the classification data into a training set and a validation set.
+ We will then train a GEE classifier using the training data, and verify its performance on the validation dataset.
+ We will also proceed to classify the rest of the image tiles to determine the burnt regions of the image, etc.

In [1]:
# ee initialisation
import ee
ee.Initialize()

In [2]:
# initial imports and parameter initialisation
import pandas as pd
import numpy as np
import xarray as xr
import re

import folium
import geehydro
import geemap


# intial parameters for the program to run
rng = np.random.default_rng(87347234) # seed the random number generator for consistency.
classification_file = '../data/classification/master_classification.csv' # file from which the classification data is to be loaded.
training_validation_ratio = 0.3 # ratio of classification data to be used in the training dataset. Keep low to avoid overfitting.

### Load in the classification data

The file was created using the produce_classification.py script.

The land classifications are:
| Classification    | Value |
|-------------------|-------|
| Peatland          | 1     |
| Burnt peatland    | 2     |
| Cleared land      | 3     |
| Agricultural land | 4     |
| Plantation        | 5     |

In [3]:
########## STEP 1: Load in the classification data
classification_data = pd.read_csv(classification_file)

coords = classification_data['coords']
classes = classification_data['classification']

# The classification data needs to be converted into a ee.FeatureCollection object.
# I will start by creating a list of ee.Feature objects. 
# These will then be shuffled and split to create the training and validation datasets.
# These ee.Feature objects can then be converted into ee.FeatureCollection objects

ee_Features = []

# generate the list of ee.Features objects 
for point in zip(coords,classes):
    # convert the string coordinate of the csv to an array
    coord = point[0].split(',')
    lon = float(coord[0][1:])
    lat = float(coord[1][:-1])
    coord = np.array([lon, lat])
    
    p = ee.Geometry.Point([*coord])
    feat = ee.Feature(p, {'landcover':point[1]})
    ee_Features.append(feat)

# randomly shuffle and then split the ee.Features list
rng.shuffle(ee_Features)
split_index = np.round( len(ee_Features) * training_validation_ratio ).astype(int)

ee_Features_training = ee_Features[:split_index]
ee_Features_validation = ee_Features[split_index:]

print('n_training', len(ee_Features_training))
print('n_validation', len(ee_Features_validation))

# create the feature collections for the training and validation data
ee_training = ee.FeatureCollection(ee_Features_training)
ee_validation = ee.FeatureCollection(ee_Features_validation)

#print(ee_training.getInfo())

n_training 267
n_validation 622


### Load in the GEE image tiles

We need to load in the GEE image tiles which we want to perform the classification on.

We will do this by laoding in the data from the [WHICH SATELLITE???] satellite and filtering the images based on date, region, cloudiness, etc.
We will display an image of the area on a folium map to check we have the correct area.

From this we can generate the NDWI and other bands that we can add to the image collection.
We'll then extract a singular image to train our classifier.

In [5]:
###############
## CREATE IMAGE COLLECTION
################
'''
def maskS2clouds(image):
    qa = image.select('QA60')
    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])
'''
ROI = geemap.geojson_to_ee('data/ROI/BogsOnFire_ROI.geojson')

def addBANDS(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B8', 'B12']).rename('NDWI')
    return image.addBands(ndvi).addBands(ndwi)

dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2017-01-01', '2022-11-01').filterBounds(ROI).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',30))
#.map(maskS2clouds);
                  
                   
dataset = dataset.map(addBANDS);

Could not convert the geojson to ee.Geometry()


Exception: string indices must be integers

In [33]:
dates = pd.date_range('2018-01-01','2022-11-01', freq = 'MS')
imgs = []

def collectYearN(dates):
    for i in range(len(dates)-1):
        start = dates[i]
        end = dates[i+1]
        #start = ee.Date.fromYMD(year,month,1)
        #end = start.advance(1,'month')
        imgs.append(dataset.filterDate(start,end).reduce(ee.Reducer.median()).set({'date':start}))

merged_years_dataset = ee.ImageCollection(imgs)

DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
               '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
               '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
               '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
               '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
      

In [ ]:
# load in the individual images and turn them into an image collection
num_images = 55

load_location = 'users/RNIng/BogsOnFire/{}'

imgs = []

for i in range(num_images):
    assetID = '{}'.format(i)
    imgs.append(ee.Image(assetID))

merged_years_dataset = ee.ImageCollection(imgs)

In [ ]:
# need to start by loading in the image collection upon which we want to perform our classification.

#ee_ImCollection = ee.ImageCollection('')

Map = folium.Map(location=[-3.893, 58.571], zoom_start=10);

merged_years_dataset = ee.ImageCollection('users/RNIng/BogsOnFire/monthly_image_timeseries')

In [30]:
'''Fran's code

Need to determine how we will load in the ee.ImageCollection object, here called merged_years_dataset
Will also need to create the Map object. Not sure if this is a folium or ee object...
'''

merged_years_dataset = ee.ImageCollection('users/RNIng/BogsOnFire/monthly_image_timeseries')

def finalCollectionFn(image):
  return image.visualize({'bands': ['B3_median', 'B2_median', 'B1_median'], 'min': 200, 'max': 5000, 'gamma': 1.2})

finalCollection = merged_years_dataset.map(finalCollectionFn)


visualization = {
  'min': 200.365,
  'max': 2965.885,
  'bands': ['B3_median', 'B2_median', 'B1_median'],
};

Map = folium.Map(location=[-3.893, 58.571], zoom_start=10);

Map.addLayer(merged_years_dataset.median(), visualization, 'RGB');
####
# CREATE/TRAIN THE CLASSIFIER
####
DatasetList = ee.ImageCollection(merged_years_dataset).toList(47);
ClassificationImage = ee.ImageCollection(ee.List(DatasetList).get(4));
print('ClassificationImage: ', ClassificationImage);
bands = ["B1_median", "B2_median", "B3_median", "B4_median",
             "B5_median", "B6_median", "B7_median", "B8_median",
             "B8A_median", "B9_median", "B11_median", "B12_median",
             "AOT_median", "WVP_median", "SCL_median", "TCI_R_median",
             "TCI_G_median", "TCI_B_median", "MSK_CLDPRB_median",
             "MSK_SNWPRB_median", "QA10_median", "QA20_median",
             "QA60_median", "NDVI_median", "NDWI_median"];
classifier = ee.Classifier.smileRandomForest({numberOfTrees: 200,
                                                  minLeafPopulation: 1,
                                                  bagFraction: 0.5});
label = 'Landcover'; #Change with name of ground data
Data = ee_training;
training = ClassificationImage.select(bands).sampleRegions({
  'collection': Data,
  'properties': [label],
  'scale': 30,
  'tileScale': 6
});

'''Random partitioning of test/validation data. I've already performed this above.
withRandom = training.randomColumn('random');
trainingPartition = withRandom.filter(ee.Filter.lt('random', 0.6));
testvalPartition = withRandom.filter(ee.Filter.gte('random', 0.6));
testvalwithRandom = testvalPartition.randomColumn('ran1dom');
testingPartition = testvalwithRandom.filter(ee.Filter.lt('ran1dom', 0.5));
validationPartition = testvalwithRandom.filter(ee.Filter.gte('ran1dom', 0.5));
'''
#print('trainingPartition: ', trainingPartition);
#print('testingPartition: ', testingPartition);
#print('validationPartition: ', validationPartition);
#Trained with X% of our data.
trainedClassifier = classifier.train(ee_training, label, bands);
#Classify the image with the same bands used for training.
trainedClassified = ClassificationImage.select(bands).classify(trainedClassifier);
####
#ACCURACY ASSESSMENT OF CLASSIFICATION
####
#Get a confusion matrix representing resubstitution accuracy.
trainedAccuracy = trainedClassifier.confusionMatrix();
print('trained error matrix: ', trainedAccuracy);
print('trained overall accuracy: ', trainedAccuracy.accuracy());
#Classify the test FeatureCollection.
tested = ee_validation.classify(trainedClassifier);
#Get a confusion matrix representing expected accuracy.
testAccuracy = tested.errorMatrix(label, 'classification');
print('Tested overall accuracy: ', testAccuracy.accuracy());

'''Exporting images, not strictly necessary at this point?
Export.table.toDrive(tested, "Malitested", "TLLGExports");
print('trainedClassified', trainedClassified);
Export.image.toDrive({
  image: trainedClassified,
  description: 'trainedClassified',
  scale: 30,
  region: ROI
});
'''
Map.addLayer(trainedClassified, {bands: ['classification'], min: 1, max: 6}, 'trainedClassifiedFor');
'''
///#VALIDATION
#Classify the test FeatureCollection.
validation = validationPartition.classify(trainedClassifier);
#Get a confusion matrix representing expected accuracy.
validationAccuracy = validation.errorMatrix(label, 'classification');
print('Validation overall accuracy: ', validationAccuracy.accuracy());
'''
####
#CLASSIFY THE IMAGE COLLECTION
####
def ClassifyIC(image):
    classified = image.classify(trainedClassifier)
    return image.addBands(classified)

  
collection = merged_years_dataset;
collectionWC = collection.map(ClassifyIC);
print('collectionWC: ', collectionWC);
####
#TIME SERIES OF NDVI/NDWI
####
'''
collection = merged_years_dataset;
NDVI = collection.select(['NDVI']);
#NDVImed = NDVI.median();
#Create palettes for display of NDVI
ndvi_pal = ['#D73027', '#F46D43', '#FDAE61', '#FEE08B', '#D9EF8B',
'#A6D96A'];
#Create a time series chart.
plotNDVI = ui.Chart.image.seriesByRegion(collection, ROI, ee.Reducer.mean(),
'NDVI_median',500,'system:time_start', 'system:index')
              .setChartType('LineChart').setOptions({
                title: 'NDVI short-term time series',
                hAxis: {title: 'Date'},
                vAxis: {title: 'NDVI'}
});
print(plotNDVI);
'''

SyntaxError: invalid syntax (3956743669.py, line 6)